In [1]:
import pandas as pd
import numpy as np

## Data

In [123]:
sales = pd.read_csv('sales_train_validation.csv')
price = pd.read_csv('sell_prices.csv')
calendar = pd.read_csv('calendar.csv')
submission_format = pd.read_csv('sample_submission.csv')

## Preprocessing

In [124]:
#Remove 'd_' before number in column 'd'
calendar['d'] = [i.replace('d_','') for i in calendar['d']]
#Generate id in price df
price['id'] = price['item_id'] + "_" + price['store_id'] + '_validation'

## Weight for the level 12 series

In [125]:
#Use second last 28 days since we use last 28 days as test set
for day in range(1858,1886):
    wk_id = list(calendar[calendar['d']==str(day)]['wm_yr_wk'])[0]
    wk_price_df = price[price['wm_yr_wk']==wk_id]
    sales = sales.merge(wk_price_df[['sell_price','id']],on=['id'],how='inner')
    #Unit sales means the amount of money made at that day. It's trivial to see since Total money = sell_price * num of sales
    sales['unit_sales_' + str(day)] = sales['sell_price'] * sales['d_'+str(day)]
    sales.drop(['sell_price'],axis=1, inplace=True)

In [126]:
#Sum of all unit_sales
sales['dollar_sales'] = sales[[c for c in sales.columns if c.find('unit_sales')==0]].sum(axis=1)

In [127]:
#Now we drop all unit_sales columns
sales.drop([c for c in sales.columns if c.find('unit_sales')==0],axis=1,inplace = True)

In [128]:
sales['weight'] = sales['dollar_sales'] / sales['dollar_sales'].sum()
sales.drop('dollar_sales',axis=1, inplace=True)

In [129]:
sales['weight'] /= 12

In [131]:
sales.shape

(30490, 1920)

## Infer forecast for any level??

In [132]:
#Making agg_df
def make_agg_df(forecast=False):
    agg_df = pd.DataFrame(sales[[c for c in sales.columns if c.find('d_')==0]].sum()).transpose()
    if forecast:
        agg_df = pd.DataFrame(sales[[c for c in sales.columns if c.find('d_')==0 or c.find('F_')==0]].sum()).transpose()
    id_cols = ['item_id','dept_id','cat_id','store_id','state_id']
    for col in id_cols:
        agg_df[col] = 'all'
    agg_df['level']=1
    agg_df['weight']=1/12
    column_agg = agg_df.columns
    
    level_groupings = {2:['state_id'],3:['store_id'],4:['cat_id'],
                   5:['dept_id'],6:['state_id','cat_id'],
                   7:['state_id','dept_id'],8:['store_id','cat_id'],
                   9:['store_id','dept_id'],10:['item_id'],
                   11:['item_id','state_id']}
    
    #Automate the process of appending time series of different level of aggs into agg_df
    for level in level_groupings:
        df = sales.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
        df['level'] = level
        for c in column_agg:
            if c not in df.columns:
                df[c] = 'all'
        agg_df = agg_df.append(df[column_agg])
    del df
    
    return agg_df

In [133]:
agg_df = make_agg_df()

In [134]:
print(sales.shape[0],agg_df.shape[0],sales.shape[0]+agg_df.shape[0])

30490 12350 42840


In [135]:
agg_df['weight'].sum() + sales['weight'].sum()

1.0000000000000007

## Top down approach

In [136]:
#Calculate the last 28 day's mean as I did in another notebook
sales['last28_mean'] = sales[[c for c in sales.columns if c.find('d_')==0 and\
    int(c.split('_')[1]) in range(1858,1886)] + ['id']].set_index('id').transpose().mean().reset_index()[0]

In [97]:
for l in range(1,10):
    #1. make forecast for this level
    this_level_df = agg_df[agg_df['level']==l].copy()
    for d in range(1,29):
        this_level_df['F_'+str(l)+"_"+str(1885+d)] = this_level_df['d_'+str(1885+d-28)]
    
    #Distribute foreast to all level 12 series
    #Find coluns doens't contain 'all'
    important_column_ids = list(this_level_df[id_cols].columns[this_level_df[id_cols].nunique()!=1])
    this_level_df.reset_index(drop=True,inplace=True)
    for i, row in this_level_df.iterrows():
        if len(important_column_ids) == 0:
            level_mean_with_cond = this_level_df[[c for c in sales.columns if c.find('d_')==0 and \
                                                 int(c.split('_')[1]) in range(1858,1886)]].transpose().mean()[0]
            proportion = sales['last28_mean'] / level_mean_with_cond
            for d in range(1,29):
                sales['F_' + str(l) + "_" + str(1885 + d)] = list(this_level_df['F_' + str(l) + '_' + str(1885+d)])[0] * proportion
        else:
            cond = True
            for col in important_column_ids:
                cond = cond & (sales[col] == row[col])
            level_mean_with_cond = this_level_df[[c for c in sales.columns if c.find('d_')==0 and \
                                                 int(c.split('_')[1]) in range(1858,1886)]].transpose().mean()[i]
            proportion = sales['last28_mean'] / level_mean_with_cond
            for d in range(1,29):
                sales.loc[cond,'F_' + str(l) + "_" + str(1885 + d)] = list(this_level_df['F_' + str(l) + '_' + str(1885+d)])[0] * proportion
#remake agg_df
agg_df = make_agg_df(forecast=True)

## RMSSE Calculation

In [98]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    assert axis == 0 or axis == 1
    if axis == 1:
        #Using axis = 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    numerator = ((ground_truth - forecast) ** 2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:,1:]-train_series[:,:-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:]-train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [99]:
#When calling .find(a) == 0 that means a has been found
train_series_cols = [c for c in sales.columns if c.find('d_')==0][:-28]
ground_truth_cols = [c for c in sales.columns if c.find('d_')==0][-28:]
forecast_cols_dict = {}
for i in range(1,10):
    forecast_cols_dict[i] = [c for c in sales.columns if c.find('F_'+str(i)+'_')==0]

In [100]:
for i in range(1,10):
    sales['rmsse_'+str(i)] = rmsse(np.array(sales[ground_truth_cols]),np.array(sales[forecast_cols_dict[i]]),np.array(sales[train_series_cols]))
    agg_df['rmsse_'+str(i)] = rmsse(np.array(agg_df[ground_truth_cols]),np.array(agg_df[forecast_cols_dict[i]]),np.array(agg_df[train_series_cols]))

In [101]:
for i in range(1,10):
    sales['wrmsse_'+str(i)] = sales['weight'] * sales['rmsse_'+str(i)]
    agg_df['wrmsse_'+str(i)] = agg_df['weight'] * agg_df['rmsse_'+str(i)]

In [102]:
for i in range(1,10):
    print('Aggregate by level',str(i)+":")
    print(sales['wrmsse_'+str(i)].sum()+agg_df['wrmsse_'+str(i)].sum())
    print('\n')

Aggregate by level 1:
0.7526969415454309


Aggregate by level 2:
0.7705207147160125


Aggregate by level 3:
0.8513445481111834


Aggregate by level 4:
0.7582245698757156


Aggregate by level 5:
1.135816499618113


Aggregate by level 6:
0.8066315193102094


Aggregate by level 7:
1.149955673701807


Aggregate by level 8:
0.8645618277353193


Aggregate by level 9:
1.3224991470261527




### Submission file generation

In [103]:
def sub_format(df):
    #Rename columns
    sub_cols = [f'F{i}' for i in range(1,29)]
    df.columns = sub_cols
    
    #Required ids
    validation_ids = sales['id'].values
    evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
    ids = np.concatenate([validation_ids, evaluation_ids])
    
    predictions = pd.DataFrame(ids, columns=['id'])
    forecast = pd.concat([df] * 2).reset_index(drop=True)
    predictions = pd.concat([predictions, forecast], axis=1)
    predictions = predictions.set_index('id')
    return predictions

In [259]:
forecast = sales[[c for c in sales.columns if c.find('F_')==0]]

In [260]:
same_as_last_28 = sub_format(forecast)

In [261]:
same_as_last_28.to_csv('last_28.csv')

In [106]:
sales['last28_mean'] = sales[[c for c in sales.columns if c.find('d_')==0 and\
    int(c.split('_')[1]) in range(1886,1914)] + ['id']].set_index('id').transpose().mean().reset_index()[0]

In [107]:
level_1_mean = agg_df[[c for c in sales.columns if c.find('d_')==0 and\
                      int(c.split('_')[1]) in range(1886,1914)]].transpose().mean().reset_index()[0][0]

In [110]:
submit_df = sales[['id']]
for i in range(1,29):
    proportion = sales['last28_mean'] / level_1_mean
    submit_df['F'+str(i)] = agg_df[agg_df['level']==1]['d_'+str(1885+i)][0] * proportion

d:\python demo\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [117]:
submit_df.drop(['id'],axis=1,inplace=True)

d:\python demo\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [119]:
arima_td = sub_format(submit_df)

In [121]:
arima_td.to_csv('AR_td.csv')